In [90]:
from pandas_datareader import data as wb
import os

for s in open_positions:
    d = wb.DataReader(s, data_source='yahoo', start='1997-1-1')
    file_path = os.path.join('data', s+'.csv')
    d.to_csv(file_path)

In [104]:
# Manual input
current_rate = 1.3265         # USD -> SGD
remaining_usd_cash = 1371.55  # Remaining USD cash in IB and DBS

In [105]:
%load_ext autoreload
%autoreload 2

from transactions import transactions
from dividends import dividends, dbs_promotes
from stocks import stocks

import pandas as pd

open_positions = {}
for t in transactions:
    if t['tick'] in open_positions:
        o_p = open_positions[t['tick']]
        o_p['transactions'].append(t)
    else:
        o_p = {'transactions': [t], 'total_cost':0, 'total_quantity':0, 'avg_cost':0, \
               'dividends':[], 'total_dividends':0, 'current_price':0, \
               'price_change':0, 'pl':0, 'pl_percent':0}
        open_positions[t['tick']] = o_p

for d in dividends:
    open_positions[d['tick']]['dividends'].append(d)

def summerize_position(p):
    for t in p['transactions']:
        p['total_cost'] += (t['buy_price']*t['quantity']+t['commission'])
        p['total_quantity'] += t['quantity']
        p['avg_cost'] = p['total_cost']/p['total_quantity']
    for d in p['dividends']:
        p['total_dividends'] += d['amount']
        
print('\nPositions in total: ' + str(len(open_positions)) + '\n')

def load_data(tick):
    file_path = os.path.join('data', tick+'.csv')
    data = pd.read_csv(file_path)
    dt_idx = pd.DatetimeIndex(data['Date'])
    data.index = dt_idx

    return data

for k, v in open_positions.items():
    summerize_position(v)
    d = load_data(k)
    v['current_price'] = d['Adj Close'].iloc[-1]
    v['price_change'] = v['current_price']/v['avg_cost']-1
    v['pl'] = v['current_price']*v['total_quantity']+v['total_dividends']-v['total_cost']
    v['pl_percent'] = (v['current_price']*v['total_quantity']+v['total_dividends'])/v['total_cost'] - 1

def get_total_value_sgd():
    total_value_sgd = 0
    for k, v in splits['SGD'].items():
        total_value_sgd += v['current_price']*v['total_quantity']
    for k, v in splits['USD'].items():
        total_value_sgd += v['current_price']*v['total_quantity']*current_rate
    return total_value_sgd

total_value_sgd = get_total_value_sgd()
    
print('\tAvg cost\tDividends\tP&L\t\tP&L%\t\tAllocation')
for k, v in open_positions.items():
    current_value = v['current_price']*v['total_quantity']
    if stocks[k]['currency'] == 'USD':
        current_value *= current_rate
    print(k + '\t' + '{:<10f}'.format(v['avg_cost']) + '\t' + str(v['total_dividends'])+ '\t\t' + \
          '{:<13f}'.format(v['pl']) + '\t' + '{:.2%}'.format(v['pl_percent']) + '\t\t' + \
          '{:.2%}'.format(current_value/total_value_sgd))
    
splits = {}
for k, v in open_positions.items():
    value = stocks[k]['currency']
    if value not in splits:
        splits[value] = {}
    splits[value][k] = v

def calculate_total_pl(category):
    total_pl = 0
    for k, v in splits[category].items():
        total_pl += v['pl']
    return total_pl
        
print('\n')
print('SG stocks PL\t\t' + str(calculate_total_pl('SGD')))
print('US stocks PL (in SGD)\t' + str(calculate_total_pl('USD')*current_rate))
print('Total PL (in SGD)\t' + str(calculate_total_pl('SGD') + calculate_total_pl('USD')*current_rate))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Positions in total: 21

	Avg cost	Dividends	P&L		P&L%		Allocation
D05.SI	20.435157 	126		6086.390534  	42.55%		6.24%
ME8U.SI	3.050636  	286.8		-966.380238  	-6.34%		4.31%
AJBU.SI	2.864771  	134.26		-372.439900  	-3.71%		2.93%
C2PU.SI	4.167125  	142.2		127.949695   	1.54%		2.56%
BABA	246.118945	-1.87		-1279.413833 	-5.71%		8.63%
AAPL	114.220645	9.24		1255.400454  	17.73%		3.40%
MSFT	208.876471	9.05		1786.349792  	25.15%		3.63%
NVDA	535.340000	0		1354.859736  	14.06%		4.49%
ARKK	103.889053	108.14		1990.179710  	20.17%		4.80%
SPY	345.785455	20.05		1581.049785  	20.78%		3.75%
ARKW	127.852174	39.48		1150.280000  	19.56%		2.86%
ARKG	76.224219 	56.37		1765.910234  	18.10%		4.68%
ARKQ	73.974583 	27.02		1062.420205  	14.96%		3.32%
ARKF	50.910625 	6.24		232.740044   	4.76%		2.09%
FB	260.923333	0		1085.580132  	15.41%		3.32%
PRNT	44.111034 	0		-1178.800283 	-11.52%		3.70%
TSLA	722.834545	0		144.440537   	0.91

In [106]:
from currency_exchange_transactions import currency_exchange_transactions

total_original_sgd_to_usd = 0
for cet in currency_exchange_transactions:
    total_original_sgd_to_usd += cet['amount']
print('Total Converted SGD to USD\t' + str(total_original_sgd_to_usd) + '\n')

usd_total = 0
for k, v in splits['USD'].items():
    usd_total += v['current_price']*v['total_quantity']
usd_total += remaining_usd_cash

from datetime import datetime, timedelta
now = datetime.now()
total_expense = 0

for t in transactions:
    dt = datetime.strptime(t['time'], '%Y-%m-%d')
    diff = now - dt
    if 'expense_ratio' in stocks[t['tick']]:        
        er = stocks[t['tick']]['expense_ratio']
        expense = t['quantity'] * er * open_positions[t['tick']]['current_price'] / 100
        total_expense += (expense * current_rate)
'''
print(usd_total)
print(usd_total*current_rate)
print(usd_total*current_rate - total_original_sgd_to_usd)
print(usd_total*current_rate - total_original_sgd_to_usd + calculate_total_pl('SGD'))
print((usd_total*current_rate - total_original_sgd_to_usd + calculate_total_pl('SGD'))/total_original_sgd)
print(total_expense)
'''
print('Actual Total PL (in SGD):\t' + str(usd_total*current_rate - total_original_sgd_to_usd + calculate_total_pl('SGD') - total_expense))
print('Actual Total PL%\t\t' + '{:.2%}'.format((usd_total*current_rate - total_original_sgd_to_usd + calculate_total_pl('SGD') - total_expense)/total_original_sgd))

Total Converted SGD to USD	263472.44175

Actual Total PL (in SGD):	14924.145035867172
Actual Total PL%		5.66%


In [112]:
# calculate ib account
transfer_to_ib = []
transfer_to_ib.append({'time':'2021-3-2', 'currency':'USD', 'amount':7571})
transfer_to_ib.append({'time':'2021-3-2', 'currency':'SGD', 'amount':14831})
transfer_to_ib.append({'time':'2021-3-4', 'currency':'SGD', 'amount':25000})
transfer_to_ib.append({'time':'2021-3-8', 'currency':'SGD', 'amount':2470})
transfer_to_ib.append({'time':'2021-4-16', 'currency':'SGD', 'amount':17218})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':30000})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':2000})

total_transfered_sgd = 0
for item in transfer_to_ib:
    if item['currency'] == 'SGD':
        total_transfered_sgd += item['amount']
print(total_transfered_sgd)

converted_sgd = 0
converted_usd = 0
for item in currency_exchange_transactions:
    if item['place'] == 'ib' and item['currency'] == 'SGDUSD':
        converted_sgd += item['amount']
        converted_usd += item['amount']/item['rate']
        
stock_sgd_purchase = 0      

for item in transactions:
    if item['currency'] == 'SGD' and item['commission'] < 5:
        stock_sgd_purchase = stock_sgd_purchase + item['buy_price']*item['quantity'] + item['commission']

print(converted_sgd)
print(total_transfered_sgd-converted_sgd-stock_sgd_purchase) # this number should be very close

# in USD
print(converted_usd+stock_sgd_purchase/current_rate)

91519
89681.44175000001
1.0582499999873107
68783.9996213532


In [115]:
# Simulation
from common import get_price

monthly_new_invest = 15000
start_td = datetime.strptime('2007-3-1', '%Y-%m-%d')

def simulate_stock(tick, start_td):
    print(tick)
    end_td = datetime.now()
    month_cnt = 0
    td = start_td
    d = load_data(tick)
    first_td = start_td = datetime.strptime(d['Date'].iloc[0], '%Y-%m-%d')
    if first_td > td:
        td = first_td

    simulated_transactions = []

    while td < end_td:
        price = get_price(d, td.strftime('%Y-%m-%d'))
        if not price:
            break
        simulated_transactions.append({'date': td, 'price': price, 'quantity': round(monthly_new_invest/price)})
        month_cnt += 1
        td += timedelta(days=30.4)

    total_money_for_invest = monthly_new_invest * month_cnt
    print(total_money_for_invest)

    market_value = 0
    total_quantity = 0
    total_cost = 0
    for st in simulated_transactions:
        total_quantity += st['quantity']
        total_cost += st['quantity']*st['price']
    market_value = total_quantity * d['Adj Close'].iloc[-1]
    print(total_cost)
    print(market_value)
    print(market_value/total_cost-1)
    print((market_value/total_cost-1)/(month_cnt/12))

for s in splits['USD']:   
    simulate_stock(s, start_td)

BABA
1200000
1200201.1274490356
2177606.657180786
0.8143681149585107
0.12215521724377659
AAPL
2550000
2549955.8728220463
30606291.188903816
11.00267483649892
0.7766594002234533
MSFT
2550000
2550458.6509218216
20328960.124877933
6.970707589213557
0.4920499474738982
NVDA
2550000
2550905.2576036453
73119324.5258789
27.664069082114082
1.952757817560994
ARKK
1170000
1169938.9917545319
4924249.478515626
3.208979710241845
0.4936891861910531
SPY
2550000
2550366.8027648926
7574249.322509766
1.9698666538077596
0.13904941085701833
ARKW
1185000
1185285.2449588776
5826160.0
3.9154075145870335
0.5947454452537266
ARKG
1170000
1169963.4877319336
4276280.547409058
2.6550547023471345
0.40846995420725146
ARKQ
1185000
1184907.3287315369
3788602.5754852295
2.197383021962563
0.3337796995386172
ARKF
405000
404916.7509441376
787064.2567634583
0.943768083015271
0.4194524813401205
FB
1620000
1620080.759595871
6312588.292358398
2.8964651946937963
0.32182946607708846
PRNT
870000
869979.1855888367
1456521.49444580